## Notebook to create brain state files from EF1_ALPHA excel sleep scoring spreadsheet

In [1]:
import os 
import numpy as np
import pandas as pd

In [403]:
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA')
epoch_number = "A, E, I, M, Q, U, Y, AC, AG, AK, AO, AS, BA, BE, BI, BM, BQ, BU, BY, CC, CG, CK, CO, CS, CW"
cols = "B, F, J, N, R, V, Z, AD, AH, AL, AP, AT, BB, BF, BJ, BN, BR, BV, BZ,CD, CH, CL, CP, CT, CX"
cols_discard = "C, G, K, O, S, W, AA, AE, AI, AM, AQ, AU, BC, BG, BK, BO, BS, BW, CA, CE, CI, CM, CQ, CU, CY"
skiprows = 5
raw_excel = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols, header= 1)

In [317]:
#list of animal ids
column_names = list(raw_excel)
animal_ids = [anim_id[0:8] for anim_id in column_names]

In [319]:
len(animal_ids)

25

In [404]:
raw_epochs = pd.read_excel('grant_EEG_classification.xlsx', usecols = epoch_number, header = 1, skiprows = 6)
raw_epochs_letter = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols, header = 1, skiprows = 6)
raw_discard = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols_discard, header = 1, skiprows = 6)

In [420]:
two_files = []
for epoch in raw_epochs:
    start_epochs = raw_epochs[raw_epochs[epoch] == 0].index.tolist()
    two_files.append(start_epochs)

In [438]:
two_files

[[0, 912],
 [0, 2391],
 [0, 2442],
 [0],
 [0, 1130],
 [0, 1476],
 [0, 2442],
 [0],
 [0, 2442],
 [0],
 [0],
 [0, 1521],
 [0],
 [0],
 [0],
 [0, 912],
 [0, 2391],
 [0],
 [0, 1521],
 [0],
 [0, 1521],
 [0],
 [0],
 [0, 2442],
 [0]]

In [482]:
for file, epoch, animal_id, epoch_letter_col, discard_col in zip(two_files, raw_epochs, animal_ids, raw_epochs_letter, raw_discard):
    if len(file) == 2:
        start_2 = file[1:2]
        start_2 = start_2[0]
        end_epoch = start_2 - 1
        epochs_letter = raw_epochs_letter[epoch_letter_col]
        drop_na = epochs_letter.dropna()
        length_col = len(drop_na) 
        epoch_column_raw = raw_epochs[epoch]
        part_1 = epoch_column_raw[0: end_epoch].astype(int)
        part_2 = epoch_column_raw[start_2: length_col].astype(int)
        epochs_letter_part_1 = epochs_letter[0:end_epoch]
        epochs_letter_part_2 = epochs_letter[start_2: length_col]
        discard_data = raw_discard[discard_col]
        discard_part_1 = discard_data[0: end_epoch]
        discard_part_2 = discard_data[start_2: length_col]
        part_1_dict = {'epoch_numbers': part_1, 'brain_state': epochs_letter_part_1,
                       'epoch_discard_numbers': discard_part_1}
        part_2_dict = {'epoch_numbers': part_2, 'brain_state': epochs_letter_part_2,
                      'epoch_discard_numbers': discard_part_2}
        df_1 = pd.DataFrame(data = part_1_dict)
        df_2 = pd.DataFrame(data = part_2_dict)
        os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
        df_1.to_csv(str(animal_id) +'_part_1.csv', index=True) 
        df_2.to_csv(str(animal_id) +'_part_2.csv', index=True)
    else:
        epochs_letter = raw_epochs_letter[epoch_letter_col]
        drop_na = epochs_letter.dropna()
        length_col = len(drop_na) 
        epoch_column_raw = raw_epochs[epoch]
        all_epochs = epoch_column_raw[0:length_col].astype(int)
        all_epochs_letter = epochs_letter[0:length_col]
        discard_data = raw_discard[discard_col]
        all_discard_epochs = discard_data[0:length_col] 
        data_dict = {'epoch_numbers': all_epochs, 'brain_state': all_epochs_letter,
                       'epoch_discard_numbers': all_discard_epochs}
        os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
        data_df = pd.DataFrame(data = data_dict)
        data_df.to_csv(str(animal_id) +'.csv', index=True)

In [484]:
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
test_check = pd.read_csv('191125_A_part_2.csv')
test_check

,Unnamed: 0,epoch_numbers,brain_state,epoch_discard_numbers
0,1130,0,W,NaN
1,1131,1,W,NaN
2,1132,2,W,NaN
3,1133,3,W,NaN
4,1134,4,W,NaN
...,...,...,...,...
2163,3293,2163,W,NaN
2164,3294,2164,W,NaN
2165,3295,2165,W,NaN
2166,3296,2166,W,NaN


In [496]:
wake_indices = test_check.loc[test_check['brain_state'] == 'W'].index.tolist()
rem_indices = test_check.loc[test_check['brain_state'] == 'R']
nonrem_indices = test_check.loc[test_check['brain_state'] == 'N']
discard_indices = test_check.loc[test_check['epoch_discard_numbers'] == 'E']

In [490]:
wake_indices_list = wake_indices.index.tolist()
rem_indices_list = rem_indices.index.tolist()
nonrem_indices_list = nonrem_indices.index.tolist()
discard_indices_list = discard_indices.index.tolist()

In [573]:
def non_match_elements(list_a, list_b):
    non_match = []
    for i in list_a:
        if i not in list_b:
            non_match.append(i)
    return non_match

new_list = non_match_elements(wake_indices_list, discard_indices_list)

## check that code works on individual animals

In [642]:
os.chdir('/home/melissa/PROJECT_DIRECTORIES/ef1_alpha_analysis')
%run preprocess.py

In [643]:
#paths 
recording_path = '/home/melissa/PREPROCESSING/EF1_ALPHA'
brain_state_path = '/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder'

In [644]:
testing_br_state

,Unnamed: 0,epoch_numbers,brain_state,epoch_discard_numbers
0,0,0,W,spyder semgent 1 0-5000
1,1,1,W,NaN
2,2,2,W,NaN
3,3,3,W,NaN
4,4,4,W,NaN
...,...,...,...,...
2452,2452,2452,W,NaN
2453,2453,2453,W,NaN
2454,2454,2454,W,NaN
2455,2455,2455,W,NaN


In [649]:
testing_npy = load_npy_recordings(animal_id = '191104', folder_path = recording_path, letter = 'B', recording_number = 1)
testing_br_state = load_brain_state_file(animal_id = '191104', folder_path = brain_state_path, letter = 'B', recording_number = 1)
indices_br_state = remove_E_epochs(testing_br_state, brain_state_letter = 'W')
epoch_indices = get_epoch_indices(indices_br_state)
timevalues_array = create_epoch_bins(testing_br_state, epoch_indices)

In [653]:
os.chdir('/home/melissa/PROJECT_DIRECTORIES/taini_main/scripts')
%run preproc3_filter.py

In [658]:
for column in testing_npy.T:
    filter_test = Filter(column, timevalues_array)
    print_filter = filter_test.butter_bandpass()

In [661]:
print_filter[0].shape

(1252,)

In [632]:
testing_npy.shape

(12329984, 4)

In [588]:
epoch_indices = get_epoch_indices(new_list)
print(epoch_indices)

[[0, 56], [58, 767], [769, 1152], [1155, 1168], [1170, 1295], [1297, 1371], [1381, 1385], [1389, 1436], [1441, 1443], [1453, 1455], [1461, 1468], [1471, 1478], [1480, 1497], [1499, 1515], [1527, 1528], [1600, 1606], [1716, 1716], [1929, 1931], [1934, 1934], [1938, 1938], [1942, 1974], [1983, 1989], [1992, 1993], [1996, 2005], [2007, 2039], [2042, 2050], [2053, 2054], [2058, 2061], [2065, 2090], [2096, 2097], [2102, 2115], [2118, 2167]]


In [594]:
timevalues_array = create_epoch_bins(test_check, epoch_indices)